# 디스크 컨트롤러

각 작업에 대해 작업이 요청되는 시점, 작업의 소요시간 을 담은 2차원 배열 jobs가 매개변수로 주어질 때, 작업의 요청부터 종료까지 걸린 시간의 평균을 가장 줄이는 방법으로 처리하면 평균이 얼마가 되는지 return 하도록 solution 함수를 작성해주세요. (단, 소수점 이하의 수는 버립니다)

### 제한 사항
- jobs의 길이는 1 이상 500 이하입니다.
- jobs의 각 행은 하나의 작업에 대한 작업이 요청되는 시점, 작업의 소요시간 입니다.
- 각 작업에 대해 작업이 요청되는 시간은 0 이상 1,000 이하입니다.
- 각 작업에 대해 작업의 소요시간은 1 이상 1,000 이하입니다.
- 하드디스크가 작업을 수행하고 있지 않을 때에는 먼저 요청이 들어온 작업부터 처리합니다.

In [1]:
# timing decorator function
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print ('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

In [2]:
def sequential(jobs):
    sequential_timer = 0
    sequential_times = []    
    sequential_avg = 0   
    # O(N) 
    for job in jobs:
        request_time = job[0]
        process_time = job[1]
        
        diff = job[0] - sequential_timer

        # 이전 작업을 완료한 시간보다 이후에 다음 작업이 시작된다면, 대기 시간만큼 더함.        
        if diff > 0 :
            sequential_timer += diff
        # 작업 소요시간 더해줌        
        sequential_timer += process_time

        sequential_times.append(sequential_timer-request_time)
    sequential_avg = round(sum(sequential_times)/ len(sequential_times))

    return sequential_avg

### 생각 1

1. 먼저 들어온 순서대로 처리하는 방법으로 계산해보고
2. 순서를 변경하여 계산해본다. => 이전 작업이 끝나는 시점과 가장 가까운 요청 시간으로 들어온 작업을 시작시킨다 (timer 값을 지정하여, 타임라인을 확인.)  
    1. timer 값과 가장 가까운 시기에 요청이 들어온 작업들을 우선적으로 시작시킨다.
    2. 우선 timer 보다 작거나 같은 값들을 대상으로 찾아보고
    3. 없다면, timer 보다 큰 값들을 대상으로 찾아본다.
    4. 해당 작업이 찾아지면, 해당 작업을 시작한다.
    5. timer를 해당 작업이 끝나는 시점으로 이동시킨다. (timer + 작업 소요시간)
    6. 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
3. 두 값을 비교한다


# 시도 1 : 실패 (test case 5개 통과)

In [14]:
import heapq

@timing
def solution(jobs):
    answer = 0
    
    sequential_avg = sequential(jobs) 

    # 순서를 변경하여 계산해보기
    timer = 0
    times = []
    avg = 0
    jobs_copy = jobs.copy()     
    while len(times) < len(jobs):
        # 현재 timer 값보다 작거나 작은 값을 찾는다.       
        under_timer = [job for job in jobs_copy if job[0]<=timer]
        under_timer.sort(reverse=True)   

        # print(under_timer)
        if len(under_timer)>0:
            job = under_timer[0]
            jobs_copy.remove(job)
            # timer를 해당 작업이 끝나는 시점으로 이동시킨다. (timer + 작업 소요시간)
            timer += job[1]
            # 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
            times.append(timer - job[0])
            
            # print(job, timer, times)
        else :
            # 현재 timer 값보다 큰 값을 찾는다.
            upper_timer = [job for job in jobs_copy if job[0]>timer]
            upper_timer.sort() 

            #print(upper_timer)
            
            job = upper_timer[0]
            jobs_copy.remove(job)
            
            # 작업 시작 전까지 기다렸던 시간을 더해준다.
            diff = job[0] - timer
            timer += diff
            # timer를 해당 작업이 끝나는 시점으로 이동시킨다.
            timer += job[1]
             # 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
            times.append(timer - job[0])
            #print(job, timer, times)

    avg = round(sum(times)/ len(times))

    #print(sequential_avg, avg)
    
    answer = avg

    if sequential_avg < avg:
        answer = sequential_avg        
            
    return answer

In [15]:
solution([[0, 3], [1, 9], [2, 6]])

func:'solution' args:[([[0, 3], [1, 9], [2, 6]],), {}] took: 0.0000 sec


9

# 시도 2 : 실패
1. 요청시간이 timer 값보다 작은 경우 (timer - 요청시간) + 작업시간   
2. 요청시간이 timer 값보다 큰 경우 작업시간   
1번과 2번을 통해 나온 값을 기준으로 가장 작은 값인 작업을 우선순위로.


In [37]:
import heapq

@timing
def solution(jobs):
    answer = 0

    sequential_avg = sequential(jobs) 
    # 순서를 변경하여 계산해보기
    timer = 0
    times = []
    avg = 0

    jobs_copy = jobs.copy()
    
    while len(jobs_copy)>0:

        h = []
        
        # 현재 시점에서 가장 소요시간이 적게 걸리는 작업 찾기
        for job in jobs_copy:
            request_time = job[0]
            process_time = job[1]
            v = 0
            if request_time < timer:
                v += (timer-request_time) + process_time
            else:
                v += process_time
            h.append([v,job])
        h = sorted(h,key= lambda el : el[0])
        
        job = h[0][1]

        # 해당 작업 제거
        jobs_copy.remove(job)
        
        diff = 0
        # 작업 시작 전까지 기다렸던 시간 계산
        if timer < job[0]:            
            diff = job[0] - timer    
                
        timer += diff
        # timer를 해당 작업이 끝나는 시점으로 이동시킨다.
        timer += job[1]
        # 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
        times.append(timer - job[0])

    avg = sum(times)// len(times)

    answer = avg

    if sequential_avg < avg:
        answer = sequential_avg        
            
    return answer

In [38]:
solution([[0, 3], [1, 9], [2, 6]])

func:'solution' args:[([[0, 3], [1, 9], [2, 6]],), {}] took: 0.0000 sec


9

# 시도 3 : 성공
작업이 끝나는 시점을 timer 하나로 관리하지 않고 작업이 시작되고 끝나는 지점을 각각 지정한다.   
그리고 그 기간 사이에 요청이 들어온 작업 중 가장 처리 시간이 짧은 작업을 먼저 처리해보자.   

In [39]:
import heapq

@timing
def solution(jobs):
    answer = 0
    sequential_avg = sequential(jobs)
  
    # start 를 0으로 놓고 아래 start <= job[0] <= end 이런식으로 하면,  
    # start 지점을 옮길 때, 0으로 옮긴다면, 0초에 들어온 작업이 두 번째 반복에서 다시 포함된다. 
    start = -1
    end = 0    
    request_btw = []

    # 모든 작업을 수행할 때까지 진행
    count = 0
    while count < len(jobs):
        for job in jobs:
            # start 지점과 end 지점 을 기준으로 사이에 요청이 들어온 작업이 있는지 확인해본다.
            if start < job[0] <= end:
              
                request_time = job[0]
                process_time = job[1]
                
                # 대기 시간이 있었다면 대기한 시간만큼 시간을 추가해준다.
                answer += (end - request_time)
                
                # 그리고 작업 시간을 집어넣는다.
                heapq.heappush(request_btw,process_time)

        # 들어온 작업이 있다면 
        if len(request_btw) > 0:
            num_jobs = len(request_btw)
            job_process_time = heapq.heappop(request_btw)
            # 이제 수행될 작업의 시간만큼 다른 작업들이 기다려야 한다. 
            answer += num_jobs * job_process_time
            # start 지점을 옮겨준다. (작업을 하나 했으니까)
            start = end
            # end 지점도 옮겨준다.
            end += job_process_time

            count+=1
    
        else:
            # 하나도 걸린게 없다면, 끝 지점을 1씩 증가하여 확인해본다.
            end+=1
                                 
    # 이렇게 하면 오답임! round 는 말 그대로 반올림을 해주는거네           
    # answer = round(answer/len(jobs))
    
    # 소수점 버림 하려면 이렇게
    answer = answer//len(jobs)
    if sequential_avg < answer:
        answer = sequential_avg        
            
    return answer

In [40]:
solution([[0, 3], [1, 9], [2, 6]])

func:'solution' args:[([[0, 3], [1, 9], [2, 6]],), {}] took: 0.0000 sec


9